<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js004_fastLJMD2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js004_fastLJMD2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023 Mitsuru Ikeuchi
#  home page: http://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.08.16 created,  last updated on 2023.12.19
#


# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js004_fastLJMD2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js004_fastLJMD2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.04.25 created, last updated on 2018.011.11
//    ver 0.0.1  2019.01.09 v1, last updated on 2021.04.23
//    ver 0.0.2  2021.10.27 v2, last updated on 2021.10.27
//    ver 0.0.3  2023.02.18 v3, last updated on 2023.08.11
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), near means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const fastLJMD2D = (function(){ // ====================  fastLJMD2D Module  ====================

	const g_Pi = Math.PI;					// Math.PI ~ 3.141592653589793
	const g_AMU = 1.66053904e-27;			// (kg) atomic mass unit
	const g_kB = 1.380649e-23;				// (J/K) Boltzmann's constant
	const g_nMax = 1000;					// array max

	let g_molecKind = 2;					// molecule kind: 0:He, 1:Ne, 2:Ar, 3:Kr, 4:Xe, 5:Hg
	let g_nParticles = 200;					// number of particles
	let g_sysTime = 0.0;					// (s) system time
	let g_timeStep = 5.0*1.0e-15;			// (s) time step
	let g_xMax = 6.0E-9;					// (m) x-Box size
	let g_yMax = 6.0E-9;					// (m) y-Box size
	let g_kineticEnergy = 0.0;				// (J) total kinetic energy
  	let g_potentialEnergy = 0.0;			// (J) total potential energy
	let g_rCutoff = 1.0e-9;					// (m) force cutoff length
	let g_hh = 1.0e-12;						// forceTable r-division

	const g_xx = dim1( g_nMax );			// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );			// (m) y-component of i-th particle position
	const g_vx = dim1( g_nMax );			// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );			// (m/s) y-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );			// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );			// (N) y-component of total force applied i-th particle
	const g_potentialTable = dim1( 1010 );	// (J) potential table [V[0], V[hh], V[2hh],..., V[rCutoff/hh] ]
	const g_forceTable = dim1( 1010 );		// (N) force table [F[0], F[hh], F[2hh],..., F[rCutoff/hh] ]
	const g_reg = dimInt2( g_nMax, 100 );	// reg[i][0]:total number of particles near i-th particle
											// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	// ----- material data

	let g_mass = 39.948*g_AMU;			// (kg) mass of Ar
	let g_sigma = 3.418e-10;			// (m) Lennard-Jones potential sigma for Ar
	let g_epsilon = 1.711e-21;			// (J) Lennard-Jones potential epsilon FOR Ar


	// --------------------  Lennard-Jones potential data  --------------------

	const g_LJMolec = [
	//    mass(kg)        E(J)         sigma(m)   string  color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[ 200.590*g_AMU, 851.0*g_kB, 2.898e-10, "Hg",  "#8800ee" ], // 5 Hg
		[   2.016*g_AMU,  33.3*g_kB, 2.968e-10, "H2",  "#ff8800" ], // 6 H2
		[  28.013*g_AMU,  91.5*g_kB, 3.681e-10, "N2",  "#ff0000" ], // 7 N2
		[  31.999*g_AMU, 113.0*g_kB, 3.433e-10, "O2",  "#ff0088" ], // 8 O2
		[  18.015*g_AMU, 809.1*g_kB, 2.641e-10, "H2O", "#ff00ff" ], // 9 H2O
		[  16.043*g_AMU, 137.0*g_kB, 3.822e-10, "CH2", "#bb00ff" ], //10 CH4
		[  44.010*g_AMU, 190.0*g_kB, 3.996e-10, "CO2", "#6600ff" ], //11 CO2
		[  28.011*g_AMU, 110.0*g_kB, 3.590e-10, "CO",  "#0000ff" ]  //12 CO
	];

	const g_rCollision = [];	// (m) = sigma, V(sigma)==0.0
	const g_rBond = [];			// (m) bond length == 2.0*g_rBond[kind], == sigma*2^(1/6)
	const g_massOf = [];		// (kg) g_massOf[kind] : mass of kind
	const g_strOf = [];			// g_strOf[kind] : string of kind, such as "Ar"
	const g_colorOf = [];		// g_colorOf[kind] : color of kind

	(function() {
		const n = g_LJMolec.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_LJMolec[i][2]*0.5;
			g_rBond[i] = g_LJMolec[i][2]*0.5*1.12246;
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
		}
	}());

	// ----- set potentialTable[ir] and forceTable[ir]

	function setForceTable() {

		for (let ir=1; ir<=1001; ir++) {
			const r = ir*g_hh;
			const ri = (g_sigma/r);
			const r6 = ri*ri*ri*ri*ri*ri;
			// V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
			g_potentialTable[ir] = cutoff(r)*4.0*g_epsilon*r6*(r6-1.0);
			//forceTable[ir] = cutoff(r)*(24.0*epsilon*r6*(2.0*r6-1.0)/r);
		}
		g_potentialTable[0] = g_potentialTable[1] + g_potentialTable[2];

		for (let ir=1; ir<=1000; ir++) {
			// force(r) = -grad V(r)
			g_forceTable[ir] = -(g_potentialTable[ir+1] - g_potentialTable[ir-1])/(2.0*g_hh);
		}
		g_forceTable[1001] = -(0.0 - g_potentialTable[1000])/(2.0*g_hh);
		g_forceTable[0] = g_forceTable[1];
	}

	function cutoff(r) {
		let ret;

		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(g_Pi*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( kind, nn, BoxSizeInNM, contTemp ) {

		g_sysTime = 0.0;
		g_molecKind = kind
		g_mass = g_LJMolec[g_molecKind][0];
		g_epsilon = g_LJMolec[g_molecKind][1];
		g_sigma = g_LJMolec[g_molecKind][2];
		setForceTable();
		g_xMax = BoxSizeInNM*1.0e-9;
		g_yMax = BoxSizeInNM*1.0e-9;
		g_nParticles = setParticles(nn);
		ajustVelocity(contTemp);
	}

	function setParticles( nn ) {
		const sigma = g_sigma;
		let nParticles = nn;

		for ( let j=0; j<nn; j++ ) {
			let loopCount = 0;
			let i;
			do {
				g_xx[j] = (g_xMax-1.2*sigma)*Math.random() + 0.6*sigma;
				g_yy[j] = (g_yMax-1.2*sigma)*Math.random() + 0.6*sigma;
				for ( i=0; i<j; i++ ) {
					if ((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]) < 1.4*sigma*sigma) break;
				}
				loopCount = loopCount + 1
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nParticles = j - 1;
				break;
			}
		}
		for ( let i=0; i<nParticles; i++ ) {
			g_vx[i] = 200.0*normalRandom3();
			g_vy[i] = 200.0*normalRandom3();
			g_ffx[i] = 0.0;
			g_ffy[i] = 0.0;
		}
		return nParticles;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {
		// tempMode  0:adiabatic, 1:controled

		if (tempMode==1) ajustVelocity(contTemp);
		registerNearParticles();
		for (let i=1; i<=20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
	}

	function moveParticles(dt) {
		const nn = g_nParticles;
		const a = 0.5*dt/g_mass;
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
		}

		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
	}

	function calcForce() {
		const nn = g_nParticles, s05 = 0.5*g_sigma;
		g_potentialEnergy = 0.0;
		for(let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0;
		}

		for(let i=0; i<nn-1; i++) {
			for(let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j];
				const yij=g_yy[i]-g_yy[j];
				const rij = Math.sqrt(xij*xij+yij*yij);
				if (rij<g_rCutoff) {
					const f = force(rij);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
				}
			}
		}
		for(let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
		}
	}

	function force(r) { // forceTable - linear interporation

		const ir = Math.floor(r/g_hh);
		const a = r - ir*g_hh;
		g_potentialEnergy += ((g_hh-a)*g_potentialTable[ir] + a*g_potentialTable[ir+1])/g_hh;
		return ((g_hh-a)*g_forceTable[ir] + a*g_forceTable[ir+1])/g_hh;
	}

	function boundaryForce(r) {

		const ri = (g_sigma/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*g_epsilon*r6*(r6-1.0);
		return (24.0*0.5*g_epsilon*r6*(2.0*r6-1.0)/r);
	}

	// --- registeration

	function registerNearParticles() {
		const nn = g_nParticles;
		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn = g_nParticles;
		let mx = 0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek = 0.0; // sum up kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mass*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]);
		}
		return ek/(nn*g_kB);
	}

	function ajustVelocity( temp ) {
		const nn = g_nParticles, a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition(kind,nn,BoxSizeInNM,contTemp)
		evolve:			timeEvolution,			// timeEvolution(tempMode,contTemp)

		setTemp:		ajustVelocity,			// ajustVelocity( contTemp )

		getSysParam:	function() { return [ g_molecKind, g_nParticles, g_timeStep, g_xMax, g_yMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getMolecData:	function() { return [ g_mass, g_sigma, g_epsilon, g_strOf[g_molecKind] ]; },

		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i] ]; },
	};

})(); // ====================  end of fastLJMD2D Module  ====================


const js004 = (function(){ // ====================  js Module  ====================

	const theModule = fastLJMD2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 2;
	let v_nn = 200;
	let v_BoxSizeInNM = 12.0;
	let v_tempMode = 0;			// 0: adiabatic, 1: temp.control
	let v_contTemp = 300.0;

	let p_molecKind, p_nParticles, p_timeStep, p_xMax, p_yMax; // = theModule.getSysParam();
	let sysTime, temperature, kineticEnergy, potentialEnergy;
  let kindList = [];
  let xxList = [];
  let yyList = [];
  let vxList = [];
  let vyList = [];

	let nCalc = 2;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;
	let timeStamp = 0.0;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
		if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_nn, v_BoxSizeInNM, v_contTemp ); // ( nn, BoxSizeInNM, contTemp )
			[ p_molecKind, p_nParticles, p_timeStep, p_xMax, p_yMax ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode, v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		timeStamp = sysTime;
    for (let i=0; i<p_nParticles; i++) {
			let x, y, vx, vy;
      kindList[i] = v_theme; // kind = v_theme
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
		}
	}


	// --------------------  draw  --------------------

	const ballColor = [ /*0:He*/ '#ff5050', /*1:Ne*/ '#ffb400', /*2:Ar*/ '#b4ff00',
						/*3:Kr*/ '#50ff50', /*4:Xe*/ '#50b4ff', /*5:Hg*/ '#8000ff' ];

	function draw( ctx ) {
		[ sysTime, temperature, kineticEnergy, potentialEnergy ] = theModule.getNow();
		let mass, sigma, epsilon, molecStr, molecColor;
		[ mass, sigma, epsilon, molecStr ] = theModule.getMolecData();

		let xp = 40, yp = 5, xSize = 400, ySize = 400, yTextPos= yCanvasSize-45;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// box
		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		//drawBalls( ctx, sigma, xp, yp, scale, molecColor )
		drawBalls(ctx, sigma, xp, yp, xSize/p_xMax, ballColor[p_molecKind] );

		// caption
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, xp, yTextPos);
		ctx.fillText(`Temp = ${temperature.toFixed(1)} (K)`, xp+200, yTextPos);
		ctx.fillText(`molec: ${molecStr}, N = ${p_nParticles}`, xp, yTextPos+20);
		ctx.fillText(`cont. Temp = ${v_contTemp.toFixed(0)} (K)`, xp+200, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)} (nm)`, xp, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, xp+200, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function drawBalls( ctx, sigma, xp, yp, scale, molecColor ) {
		const r = sigma*0.5, nn = p_nParticles, yMax = p_yMax, twoPi = 2.0*Math.PI;
		ctx.fillStyle = molecColor;
		for (let i=0; i<nn; i++) {
			ctx.beginPath();
			let x, y;
			[ x, y ] = theModule.getPosition(i);
			ctx.arc(x*scale+xp, (yMax-y)*scale+yp, r*scale, 0, twoPi, false);
			ctx.fill();
		}
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 1 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setnParticles() {
		const n = 1 + document.getElementById("slct_nn").selectedIndex;
		v_nn = n*100;
		resetFlag = true;
	}

	function setTemperature(temp) {
		theModule.setTemp(temp);
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme;
    document.getElementById("slct_theme").selectedIndex = theme-1;
    resetFlag = true;
  }

	function pysetnParticles( nn ) {
		v_nn = nn;
		resetFlag = true;
	}

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, temperature, kineticEnergy, potentialEnergy ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ timeStamp, kindList, xxList, yyList, vxList, vyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setnParticles:	setnParticles,	// setnParticles()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
		pysetnParticles: pysetnParticles, // pysetnParticles( nn )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, temperature, kineticEnergy, potentialEnergy ]
		pygetParticlesList, pygetParticlesList, //() :return [ timeStamp, kindList, xxList, yyList, vxList, vyList ]
	};

})(); // ====================  js004 module end  ====================


const js = js004;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js004] fast Lennard-Jones molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>molecules:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ne</option><option selected>Ar</option><option>Kr</option>
<option>Xe</option><option>Hg</option>
</select>
    <span style="margin-right: 50px;"></span>
<label>N=</label>
<select id="slct_nn" onChange="js.setnParticles()">
<option>100</option><option selected>200</option><option>300</option>
<option>400</option><option>500</option>
</select>
    <span style="margin-right: 50px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont. Temp</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option>1</option><option selected>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and get temporally data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get temporally data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme

import time

# exec html-js code
exec_html_js()
print("-- start --")

# reset
eval_js( 'js.reset()' )
print("-- reset --")

# change theme and dispMode
themeList = [ 'He', 'Ne', 'Ar', 'Kr', 'Xe', 'Hg' ]
for theme in [ 1, 2, 3, 4, 5 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for i in range(4):
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
    energy = kineticEnergy + potentialEnergy
    print( f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
    time.sleep(2)

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# set nParticles = 500

import time

# exec html-js code
exec_html_js()
print("-- start --")

# set nParticles = 500, and reset
nParticles = 500
print("-- set number of particle = ", nParticles, " --")
eval_js( 'js.pysetnParticles({})'.format(nParticles) )

# get temporally data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# gas cool down

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# memory for plot
timeList = []
tempList = []
energyList = []

# gas cool down
for i in range(20):
  # control system temperature
  contTemp = 300.0 - 15.0*i
  eval_js('js.pysetTemperature({})'.format(contTemp))
  time.sleep(3)
  # get temporally data and print
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)
  if i%2==0:
    print( f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

# plot temperature and energy
plt.plot( timeList, tempList, 'xb-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

plt.plot( timeList, energyList, 'og-' )
plt.xlabel('time (ps)')
plt.ylabel('energy ( x10^-20 J )')
plt.show()

In [ ]:
# get particles data

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get temporally data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f' i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(1)

# get particle data
print("-- get particles data --")
[ timeStamp, kindList, xxList, yyList, vxList, vyList ] = eval_js('js.pygetParticlesList()')
print(f'time stamp = {timeStamp*1e12:>8.3f} (ps)')
print(f'len(xxList) ={len(xxList):>4d}, len(yyList) ={len(yyList):>4d}, len(vxList) ={len(vxList):>4d},  len(vyList) ={len(vyList):>4d}')
print("")

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# save particles data

import numpy as np

# convert [ kindList, xxList, yyList, vxList, vyList ] to np_data
np_data = np.array([ kindList, xxList, yyList, vxList, vyList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js004_data.npy'" )
np.save( 'js004_data.npy', np_data )


# The saved file can be viewed by clicking on the folder icon on the left edge of the notebook.
# You must download this file to save it permanently.
# Select Download from the '...'.T menu of the file to download it.

In [9]:
# Colab AI wrote:
# prompt: save np_data.T as csv file

np.savetxt('js004_data.csv', np_data.T, delimiter=',')


In [ ]:
# plot particle position

import numpy as np
import matplotlib.pyplot as plt

# load np_data <-- np.array([ kindList, xxList, yyList, vxList, vyList ])
print("-- load data --")
loaded_data = np.load('js004_data.npy')

# preparation plot data
X = loaded_data[1]*1e9
Y = loaded_data[2]*1e9

# scatter plot
fig = plt.figure(figsize=(8, 8))
plt.scatter(X,Y)
plt.xlabel('x ( nm )')
plt.ylabel('y ( nm )')
plt.title('Ar in the box')
plt.show()

In [ ]:
# plot particle velocity

import numpy as np
import matplotlib.pyplot as plt

# load np_data <-- np.array([ kindList, xxList, yyList, vxList, vyList ])
print("-- load data --")
loaded_data = np.load('js004_data.npy')

# preparation plot data
U = loaded_data[3]
V = loaded_data[4]
C = (U/300)**2 + (V/300)**2 # proportional to kinetic energy of particle

# scatter plot
fig = plt.figure(figsize=(8, 8))
plt.scatter(U, V, c=C, cmap='jet')
plt.xlabel('vx ( m/s )')
plt.ylabel('vy ( m/s )')
plt.title('velocity space')
plt.colorbar
plt.show()